<a href="https://colab.research.google.com/github/gogzicole/Hamoye-Data-science/blob/master/stage_C_quiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [169]:
import pandas as pd
import numpy as np
from pandas import Series,DataFrame

**Loading Dataset and checking for missing values**

In [171]:
url = 'https://raw.githubusercontent.com/gogzicole/Hamoye-Data-science/master/Data_for_UCI_named.csv'
df = pd.read_csv(url)
df.isnull().sum()
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


**Assigning independent and dpndent variables from dataset**

In [172]:
x = df.drop(columns= ['stab','stabf'])
y = df['stabf']
y.value_counts()

unstable    6380
stable      3620
Name: stabf, dtype: int64

**Training Dataset**

In [177]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = 1, test_size=0.2 )
x_test.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4
9953,6.877876,4.113820,9.356768,8.299753,4.056779,-1.897470,-1.590581,-0.568728,0.276567,0.845536,0.112440,0.822562
3850,5.802841,6.271371,4.731540,3.819867,3.579569,-1.709480,-1.067511,-0.802579,0.077527,0.416478,0.912846,0.861306
4962,2.286998,4.385142,2.830232,5.293880,3.035814,-1.202764,-0.902011,-0.931039,0.924216,0.130186,0.703887,0.063811
3886,5.019920,2.209962,6.266080,0.578901,4.322584,-1.960207,-1.074561,-1.287815,0.546910,0.065992,0.427349,0.814648
5437,7.646145,9.187896,5.484219,9.934313,3.634226,-1.254541,-1.335366,-1.044319,0.561528,0.121611,0.787318,0.300314


In [175]:
y_test.value_counts() #there is an imbalance in the Dataset but we would work with them 

unstable    1288
stable       712
Name: stabf, dtype: int64

**Scaling Train Set**

In [190]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_x_train = scaler.fit_transform(x_train)
scaled_x_train = DataFrame(scaled_x_train,columns = x_train.columns)
scaled_x_test = scaler.transform(x_test)
scaled_x_test = DataFrame(scaled_x_test, columns = x_test.columns)
scaled_x_train.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4
0,0.367327,-0.986042,0.650447,1.547527,-0.291490,0.061535,1.293862,-0.845074,0.160918,0.339859,0.585568,0.492239
1,-0.064659,0.089437,1.035079,-1.641494,0.619865,-0.067235,-1.502925,0.486613,-0.293143,-1.558488,1.429649,-1.443521
2,-1.467850,1.298418,-0.502536,1.166046,-0.180521,0.490603,0.682560,-0.855302,1.399350,1.451534,-1.045743,0.492489
3,0.820081,0.529920,1.299657,-1.141975,-0.812854,-0.763632,1.521579,0.658780,-0.958319,1.361958,1.604140,0.275303
4,0.665424,-1.425627,0.312300,0.919137,-1.614296,0.760315,1.422019,0.639243,1.676895,0.695660,1.137504,-1.312575


**Random Forest**

In [188]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(random_state =1)
classifier.fit(scaled_x_train,y_train)
forest_pred = classifier.predict(scaled_x_test)

In [191]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test,forest_pred)
round(accuracy,4)

0.929

**Extra Trees Classifier**

In [192]:
from sklearn.ensemble import ExtraTreesClassifier
tree_classifier = ExtraTreesClassifier(random_state = 1)
tree_classifier.fit(scaled_x_train,y_train)
tree_pred = tree_classifier.predict(scaled_x_test)


In [201]:
accuracy = accuracy_score(y_test,tree_pred)
round(accuracy,4)

0.928

**XG-Boost**

In [194]:
from xgboost import XGBClassifier
boost = XGBClassifier(random_state=1)
boost.fit(scaled_x_train,y_train)
boost_pred = boost.predict(scaled_x_test)

In [196]:
accuracy = accuracy_score(y_test, boost_pred)
round(accuracy,4)

0.9195

**Light GBM**

In [197]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(random_state=1)
lgbm.fit(scaled_x_train,y_train)
lgbm_pred = lgbm.predict(scaled_x_test)


In [199]:
accuracy = accuracy_score(y_test,lgbm_pred)
round(accuracy,4)

0.9375

**Tunning Parameters**

In [200]:
n_estimators = [50,100,300,500,1000] #number of estimators
min_samples_split = [2,3,5,7,9] #minimum number of splits
min_samples_leaf = [1,2,4,6,8] #minimum number of samples for leaf node
max_features = ['auto','sqrt','log2',None] #the number of features to consider when looking for the best split
hyperparameter_grid = {'n_estimators':n_estimators,
                       'min_samples_leaf':min_samples_leaf,
                       'min_samples_split':min_samples_split,
                       'max_features':max_features}

**Randomized Search**

In [86]:
from sklearn.model_selection import RandomizedSearchCV

In [87]:
random_search = RandomizedSearchCV(estimator = tree_classifier,
                                   param_distributions=hyperparameter_grid,random_state=1)
random_search = random_search.fit(scaled_x_train,y_train)

In [88]:
random_search.best_params_

{'max_features': None,
 'min_samples_leaf': 8,
 'min_samples_split': 2,
 'n_estimators': 1000}

**Training ExtraTreesClassifier with new Hyperparameter**

In [202]:
new_classifier = ExtraTreesClassifier(random_state=1,n_estimators = 1000,
                                       min_samples_leaf= 8,max_features=None,
                                       min_samples_split=2)
new_classifier.fit(scaled_x_train,y_train)
new_pred = new_classifier.predict(scaled_x_test)

In [203]:
accuracy = accuracy_score(y_test,new_pred)
round(accuracy,4) #its value reduced 

0.927

**Feature Importance**

In [204]:
importance = random_search.best_estimator_.feature_importances_
importance

array([0.13723975, 0.1405075 , 0.13468029, 0.13541676, 0.00368342,
       0.00533686, 0.00542927, 0.00496249, 0.10256244, 0.10757765,
       0.11306268, 0.10954089])

In [205]:
sorted(zip(importance,x),reverse = True)

[(0.14050750384993677, 'tau2'),
 (0.13723974766109256, 'tau1'),
 (0.1354167630909727, 'tau4'),
 (0.13468028520386593, 'tau3'),
 (0.11306267999167334, 'g3'),
 (0.10954089174337298, 'g4'),
 (0.10757764577478764, 'g2'),
 (0.10256244080927947, 'g1'),
 (0.005429268421191957, 'p3'),
 (0.005336864710946151, 'p2'),
 (0.004962486591192238, 'p4'),
 (0.003683422151688322, 'p1')]

least important is tau2
and most important p1